In [1]:
import pandas as pd
import requests
import sqlite3
import re
from bs4 import BeautifulSoup

In [2]:
pt_london_2025_invited = "https://fabtcg.com/en/coverage/pro-tour-london/pairings/1/"
pt_london_2025 = "https://fabtcg.com/en/coverage/pro-tour-london/decklist/{}/"

In [ ]:
def get_player_list(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    pairing = soup.find_all("div", {"class":"tournament-coverage__player-hero-and-deck"})
    decklist_urls = [re.search(r"\/decklist\/(\d+)", url) for url in (str(url) for url in (player.find("a") for player in pairing))]

    output = [url.group(1) for url in decklist_urls if url is not None]

    return output


def get_decklist(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    decklist = [data.text.strip() for data in soup.find_all("td")]

    return decklist

def decklist_to_df(decklists):
    decklists = pd.DataFrame.from_records(decklists)
    decklists.index = [re.search(r"\((\d+)", name).group(1) for name in decklists[0]]

    participants = decklists.iloc[:,0:5].copy()
    participants = participants.rename(columns={0:"Name", 1:"Event Date", 2:"Event Name", 3:"Format", 4:"Hero"})

    decklists = decklists.drop([0,1,2,3,4], axis=1)

    decklists = pd.melt(decklists, ignore_index=False, value_name="import name")["import name"].dropna().to_frame()

    decklists[["Copies","Card"]] = decklists["import name"].str.split(" x ", expand=True)
    decklists = decklists.drop("import name", axis=1)

    return participants, decklists

In [ ]:
player_list = get_player_list(pt_london_2025_invited)
tournament_lists = [get_decklist(pt_london_2025.format(player_id)) for player_id in player_list]

In [134]:
tournament_df = decklist_to_df(tournament_lists)

participants = tournament_df[0]
decklists = tournament_df[1]